# Imports and config

Run this everytime

In [73]:
import requests,json
from pprint import pprint
from fuzzywuzzy import fuzz
import numpy as np

In [102]:
API_URL = "https://api.proxycrawl.com/scraper"
API_KEY = ""
PARAMS = {
    "token":API_KEY,
    "javascript":"true",
    "url":"https://www.instagram.com/"
}

assert API_KEY != ""
assert "https://www.instagram.com/" in PARAMS["url"]

# Testing the API response

## Getting the user data

In [26]:
url = "https://www.instagram.com/artbyjuliet"
PARAMS["url"] = url
assert "https://www.instagram.com/" in PARAMS["url"]
resp = requests.get(API_URL,PARAMS)

In [27]:
result = resp.json()

In [29]:
pprint(result["body"])

{'bio': {'tags': [],
         'text': '🇬🇧🇫🇷 Comics in eng/fr ✏️ Commissions: CLOSED‼️Ask permission '
                 'for repost ✉️ business inquiries by mail ⬇️'},
 'followersCount': {'text': '92.7k', 'value': '92,775'},
 'followingCount': {'text': '265', 'value': '265'},
 'igtvs': [],
 'name': 'Juliette',
 'openStories': [{'image': 'https://instagram.fvno2-1.fna.fbcdn.net/v/t51.12442-15/e35/c297.550.528.528a/s150x150/94466204_166039001533184_3854508874995469921_n.jpg?_nc_ht=instagram.fvno2-1.fna.fbcdn.net&_nc_cat=108&_nc_ohc=pR3RV6DagycAX-A2x3E&oh=46a25c6eec53752f43f886ac15d174fc&oe=5EC4A852',
                  'text': "Commissions's profile picture"},
                 {'image': 'https://instagram.fvno2-1.fna.fbcdn.net/v/t51.2885-15/s150x150/95429213_271220900586043_2157824169714876034_n.jpg?_nc_ht=instagram.fvno2-1.fna.fbcdn.net&_nc_ohc=NbRoUvoiDj0AX97K3yc&oh=4ec4f97e5bbc588938aebf059b50013e&oe=5EED32C1',
                  'text': "Twitter's profile picture"},
                 {'i

In [31]:
print(result["body"]["picture"])

https://instagram.fvno2-1.fna.fbcdn.net/v/t51.2885-19/s150x150/83923473_679724872564551_8335522487689281536_n.jpg?_nc_ht=instagram.fvno2-1.fna.fbcdn.net&_nc_ohc=Qp_hig7KNR8AX8KygVN&oh=a569351a9db2d811389bb04380259876&oe=5EEAB0AA


In [32]:
url = "https://www.instagram.com/_akhilsingh__"
PARAMS["url"] = url
assert "https://www.instagram.com/" in PARAMS["url"]
resp = requests.get(API_URL,PARAMS)

In [33]:
result2 = resp.json()

## Checking out the format

In [34]:
result2["body"]

{'username': '_akhilsingh__',
 'verified': False,
 'postsCount': {'value': '11', 'text': '11'},
 'followersCount': {'value': '337', 'text': '337'},
 'followingCount': {'value': '1,555', 'text': '1,555'},
 'picture': None,
 'name': 'Akhil Singh',
 'bio': {'text': 'A ghost that eats pizza drinks chai, loves bikes and listens to violin and pianoBike enthusiast 🏍️',
  'tags': []},
 'openStories': [],
 'posts': [],
 'igtvs': []}

In [47]:
usr = result2["body"]
pprint(usr)

{'bio': {'tags': [],
         'text': 'A ghost that eats pizza drinks chai, loves bikes and listens '
                 'to violin and pianoBike enthusiast 🏍️'},
 'followersCount': {'text': '337', 'value': '337'},
 'followingCount': {'text': '1,555', 'value': '1,555'},
 'igtvs': [],
 'name': 'Akhil Singh',
 'openStories': [],
 'picture': None,
 'posts': [],
 'postsCount': {'text': '11', 'value': '11'},
 'username': '_akhilsingh__',
 'verified': False}


In [98]:
result2["url"]

'https://www.instagram.com/_akhilsingh__/'

## Creating a dummy genuine user

In [67]:
gen_usr={}
gen_usr["name"] = "Akhil Singh the Necromancer"
gen_usr["username"] = "_akhi1singh__"
gen_usr["verified"] = True
gen_usr["bio"] = {}
gen_usr["bio"]["text"] = "A ghost that eats pizza drinks chai, loves bikes and listens to violin and piano"
pprint(gen_usr)

{'bio': {'text': 'A ghost that eats pizza drinks chai, loves bikes and listens '
                 'to violin and piano'},
 'name': 'Akhil Singh the Necromancer',
 'username': '_akhi1singh__',
 'verified': True}


## Feature engineering

In [68]:
uname_match = fuzz.ratio(gen_usr["username"],usr["username"])/100
uname_match

0.92

In [69]:
name_match = fuzz.ratio(gen_usr["name"],usr["name"])/100
name_match

0.58

In [70]:
def word_overlap_similarity(sentence1, sentence2):
    """Function to find the similarity between list of words using % of words in common between them."""
    sentence1 = filter(None,sentence1.replace(","," ").split(" "))
    sentence2 = filter(None,sentence2.replace(","," ").split(" "))
    
    sentence1 = set(sentence1)
    sentence2 = set(sentence2)
    
    if len(sentence2.union(sentence1)) != 0:
        return len((sentence2.intersection(sentence1))) / len(sentence2.union(sentence1))
    else:
        return 0

In [71]:
word_overlap_similarity(gen_usr["bio"].get("text",""),usr["bio"].get("text",""))

0.7647058823529411

In [77]:
follow_ratio = int(usr["followersCount"]["value"].replace(",",""))/int(usr["followingCount"]["value"].replace(",",""))
follow_ratio

0.21672025723472668

In [86]:
sensitivity = 2
np.exp(-sensitivity*(follow_ratio))

0.6482748544423349

## Assembling all features into one aggregation

In [94]:
def predict_impersonation(gen_usr,usr):
    l = []
    global sensitivity
    # Img similarity goes here
    # l.append(calc_sim(gen_usr["picture"],usr.get("picture")))
    
    # if genuine user is verified, append curent usr's verification status
    if gen_usr["verified"]:
        l.append(int(not usr["verified"]))
    
    # low followers/following ratios means probably impersonation
    try:    
        follow_ratio = int(usr["followersCount"]["value"].replace(",",""))/int(usr["followingCount"]["value"].replace(",",""))
    except ZeroDivisionError:
        follow_ratio = 0
    l.append(np.exp(-sensitivity*follow_ratio))
    
    # Fuzzy match name and username
    l.append(fuzz.ratio(gen_usr["name"],usr["name"])/100)
    uname_match = fuzz.ratio(gen_usr["username"],usr["username"])/100
    if uname_match>=0 and uname_match<1:
        l.append(uname_match)
    
    # Use word overlap on description
    l.append(word_overlap_similarity(gen_usr["bio"].get("text",""),usr["bio"].get("text","")))
    
    #Get back the avg
    l = np.array(l)
    return np.average(l)

In [91]:
pprint(gen_usr)

{'bio': {'text': 'A ghost that eats pizza drinks chai, loves bikes and listens '
                 'to violin and piano'},
 'name': 'Akhil Singh the Necromancer',
 'username': '_akhi1singh__',
 'verified': True}


In [92]:
pprint(usr)

{'bio': {'tags': [],
         'text': 'A ghost that eats pizza drinks chai, loves bikes and listens '
                 'to violin and pianoBike enthusiast 🏍️'},
 'followersCount': {'text': '337', 'value': '337'},
 'followingCount': {'text': '1,555', 'value': '1,555'},
 'igtvs': [],
 'name': 'Akhil Singh',
 'openStories': [],
 'picture': None,
 'posts': [],
 'postsCount': {'text': '11', 'value': '11'},
 'username': '_akhilsingh__',
 'verified': False}


In [88]:
predict_impersonation(gen_usr,usr)

0.7825961473590552

# Making legit functions

In [99]:
def clean_data(usr:dict)->dict:
    """
    Flatten the user data and bring it to normal form
    """
    data = {}
    data["description"] = usr["bio"].get("text","")
    data["followers"]   = int(usr["followersCount"].get("value","0").replace(",",""))
    data["following"]   = int(usr["followingCount"].get("value","0").replace(",",""))
    data["verified"]    = usr["verified"]
    data["post_count"]  = int(usr["postsCount"].get("value","0").replace(",",""))
    data["name"]        = usr["name"]
    data["username"]    = usr["username"]
    data["picture"]     = usr["picture"]
    
    return data

In [100]:
def get_instagram_user(user_url):
    PARAMS["url"] = user_url
    try:
        assert "https://www.instagram.com/" in PARAMS["url"]
        usr_data = requests.get(API_URL,PARAMS).json()["body"]
        usr_data = clean_data(usr_data)
        usr_data["url"] = user_url
        usr_data["source"] = "instagram"
        return usr_data
    except AssertionError as ae:
        print(f"The given url is not of an instagram user: {ae}")
        return {"url":user_url,"message":"Extraction Failed"}

In [103]:
usr = get_instagram_user("https://www.instagram.com/artbyjuliet")

In [104]:
usr

{'description': '🇬🇧🇫🇷 Comics in eng/fr ✏️ Commissions: CLOSED‼️Ask permission for repost ✉️ business inquiries by mail ⬇️',
 'followers': 92770,
 'following': 265,
 'verified': True,
 'post_count': 274,
 'name': 'Juliette',
 'username': 'artbyjuliet',
 'picture': 'https://scontent-sea1-1.cdninstagram.com/v/t51.2885-19/s150x150/83923473_679724872564551_8335522487689281536_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com&_nc_ohc=Qp_hig7KNR8AX94M4-J&oh=0058cb4530a2a7b3c59677268a2313f5&oe=5EEAB0AA',
 'url': 'https://www.instagram.com/artbyjuliet',
 'source': 'instagram'}

In [106]:
gen_usr={}
gen_usr["name"] = "Juliet"
gen_usr["username"] = "fartbyjuliet"
gen_usr["verified"] = True
gen_usr["description"] = "Artist who loves to paint stuff"
pprint(gen_usr)

{'description': 'Artist who loves to paint stuff',
 'name': 'Juliet',
 'username': 'fartbyjuliet',
 'verified': True}


In [109]:
def predict_impersonation(gen_usr,usr):
    l = []
    global sensitivity
    # Img similarity goes here
    # l.append(calc_sim(gen_usr["picture"],usr.get("picture")))
    
    # if genuine user is verified, append curent usr's verification status
    if gen_usr["verified"]:
        l.append(int(not usr["verified"]))
    
    # low followers/following ratios means probably impersonation
    try:    
        follow_ratio = usr.get("followers",0)/usr.get("following",0)
    except ZeroDivisionError:
        follow_ratio = 0
    l.append(np.exp(-sensitivity*follow_ratio))
    
    # Fuzzy match name and username
    l.append(fuzz.ratio(gen_usr["name"],usr["name"])/100)
    uname_match = fuzz.ratio(gen_usr["username"],usr["username"])/100
    if uname_match>=0 and uname_match<1:
        l.append(uname_match)
    
    # Use word overlap on description
    l.append(1-word_overlap_similarity(gen_usr.get("description",""),usr.get("description","")))
    
    #Get back the avg
    l = np.array(l)
    return np.average(l)

In [110]:
predict_impersonation(gen_usr,usr)

0.564